In [8]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [9]:
dir = '/home/jsmith48/scratch/opt_test/'

fname = 'mol2_fix'

# Molecule file
molfile = dir + fname + '.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'opt_' +  fname +'.xyz'

dt = 0.25
C = 0.0001 # Optimization convergence

ntdir = '/home/jsmith48/scratch/ccsd_extrapolation/final_train_models_3/ani-1x_dft_x8ens/'
cns = ntdir + 'rHCNO-5.2R_16-3.5A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [10]:
#tvec = np.loadtxt('/home/jsmith48/scratch/carbon_nano_tube/new_test/'+fname+'.tvec',dtype=str)
#tvec = np.array(tvec[:,1:],dtype=np.float64)

In [11]:
#tvec

In [12]:
# Load molecule
mol = read(molfile)
# L = 40.0
# mol.set_cell(([[100.0, 0, 0],
#                [0, 35.0, 0],
#                [0, 0, 35.0]]))

#mol.set_cell((tvec))


#mol.set_pbc((True, True, True))
#mol.set_pbc((False, False, False))

# Set NC
aens = ensemblemolecule_multigpu(cns, sae, nnf, Nn, [7])
#aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print(E)

spc = mol.get_chemical_symbols()
print(len(spc))

Building...
Total time: 1.2106578350067139
-27926.360622245556
46


In [13]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=3000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 12:05:43   -27926.360622        2.0155
LBFGS:    1 12:05:43   -27926.403852        3.3248
LBFGS:    2 12:05:43   -27926.472889        0.5475
LBFGS:    3 12:05:43   -27926.501722        0.4155
LBFGS:    4 12:05:43   -27926.544954        0.4134
LBFGS:    5 12:05:43   -27926.570030        0.3084
LBFGS:    6 12:05:43   -27926.590140        0.6926
LBFGS:    7 12:05:43   -27926.607377        0.4949
LBFGS:    8 12:05:43   -27926.624349        0.2332
LBFGS:    9 12:05:43   -27926.639058        0.3330
LBFGS:   10 12:05:43   -27926.647973        0.2214
LBFGS:   11 12:05:43   -27926.657847        0.1667
LBFGS:   12 12:05:43   -27926.665278        0.1609
LBFGS:   13 12:05:43   -27926.672648        0.1764
LBFGS:   14 12:05:43   -27926.680642        0.2020
LBFGS:   15 12:05:43   -27926.688264        0.1411
LBFGS:   16 12:05:43   -27926.694274        0.1515
LBFGS:   17 12:05:43   -27926.700273        0.1457
LBFGS:   18 12:05:43   -27926.70

LBFGS:  160 12:05:46   -27926.971076        0.0460
LBFGS:  161 12:05:46   -27926.971669        0.0419
LBFGS:  162 12:05:46   -27926.972154        0.0490
LBFGS:  163 12:05:46   -27926.972511        0.0298
LBFGS:  164 12:05:46   -27926.972759        0.0269
LBFGS:  165 12:05:46   -27926.972978        0.0228
LBFGS:  166 12:05:46   -27926.973182        0.0254
LBFGS:  167 12:05:46   -27926.973364        0.0212
LBFGS:  168 12:05:46   -27926.973542        0.0237
LBFGS:  169 12:05:46   -27926.973711        0.0250
LBFGS:  170 12:05:46   -27926.973882        0.0266
LBFGS:  171 12:05:46   -27926.974044        0.0180
LBFGS:  172 12:05:46   -27926.974189        0.0166
LBFGS:  173 12:05:46   -27926.974317        0.0233
LBFGS:  174 12:05:46   -27926.974451        0.0217
LBFGS:  175 12:05:46   -27926.974614        0.0228
LBFGS:  176 12:05:46   -27926.974792        0.0255
LBFGS:  177 12:05:46   -27926.974986        0.0249
LBFGS:  178 12:05:46   -27926.975181        0.0227
LBFGS:  179 12:05:46   -27926.9

LBFGS:  321 12:05:49   -27926.985845        0.0121
LBFGS:  322 12:05:49   -27926.985885        0.0129
LBFGS:  323 12:05:49   -27926.985938        0.0114
LBFGS:  324 12:05:49   -27926.985980        0.0094
LBFGS:  325 12:05:49   -27926.986008        0.0081
LBFGS:  326 12:05:49   -27926.986034        0.0075
LBFGS:  327 12:05:49   -27926.986059        0.0080
LBFGS:  328 12:05:49   -27926.986086        0.0087
LBFGS:  329 12:05:49   -27926.986114        0.0086
LBFGS:  330 12:05:49   -27926.986138        0.0099
LBFGS:  331 12:05:49   -27926.986163        0.0087
LBFGS:  332 12:05:49   -27926.986188        0.0080
LBFGS:  333 12:05:49   -27926.986209        0.0100
LBFGS:  334 12:05:49   -27926.986230        0.0093
LBFGS:  335 12:05:49   -27926.986249        0.0114
LBFGS:  336 12:05:49   -27926.986264        0.0091
LBFGS:  337 12:05:49   -27926.986291        0.0086
LBFGS:  338 12:05:49   -27926.986313        0.0105
LBFGS:  339 12:05:49   -27926.986348        0.0122
LBFGS:  340 12:05:49   -27926.9

LBFGS:  482 12:05:52   -27926.991471        0.0055
LBFGS:  483 12:05:52   -27926.991483        0.0047
LBFGS:  484 12:05:52   -27926.991488        0.0050
LBFGS:  485 12:05:52   -27926.991497        0.0044
LBFGS:  486 12:05:52   -27926.991508        0.0042
LBFGS:  487 12:05:52   -27926.991507        0.0036
LBFGS:  488 12:05:52   -27926.991511        0.0036
LBFGS:  489 12:05:52   -27926.991522        0.0041
LBFGS:  490 12:05:52   -27926.991525        0.0049
LBFGS:  491 12:05:52   -27926.991533        0.0048
LBFGS:  492 12:05:52   -27926.991535        0.0046
LBFGS:  493 12:05:52   -27926.991544        0.0046
LBFGS:  494 12:05:52   -27926.991551        0.0050
LBFGS:  495 12:05:52   -27926.991555        0.0062
LBFGS:  496 12:05:52   -27926.991566        0.0062
LBFGS:  497 12:05:52   -27926.991576        0.0065
LBFGS:  498 12:05:52   -27926.991589        0.0059
LBFGS:  499 12:05:52   -27926.991603        0.0061
LBFGS:  500 12:05:52   -27926.991610        0.0070
LBFGS:  501 12:05:52   -27926.9

LBFGS:  643 12:05:55   -27926.991890        0.0003
LBFGS:  644 12:05:55   -27926.991888        0.0004
LBFGS:  645 12:05:55   -27926.991889        0.0003
LBFGS:  646 12:05:55   -27926.991888        0.0003
LBFGS:  647 12:05:55   -27926.991889        0.0002
LBFGS:  648 12:05:55   -27926.991892        0.0002
LBFGS:  649 12:05:55   -27926.991887        0.0003
LBFGS:  650 12:05:55   -27926.991892        0.0004
LBFGS:  651 12:05:55   -27926.991887        0.0003
LBFGS:  652 12:05:55   -27926.991889        0.0004
LBFGS:  653 12:05:55   -27926.991890        0.0003
LBFGS:  654 12:05:55   -27926.991890        0.0003
LBFGS:  655 12:05:55   -27926.991892        0.0002
LBFGS:  656 12:05:55   -27926.991889        0.0003
LBFGS:  657 12:05:55   -27926.991886        0.0002
LBFGS:  658 12:05:55   -27926.991890        0.0002
LBFGS:  659 12:05:55   -27926.991887        0.0002
LBFGS:  660 12:05:55   -27926.991887        0.0003
LBFGS:  661 12:05:55   -27926.991889        0.0002
LBFGS:  662 12:05:55   -27926.9

In [14]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev
    Fstddev =  hdt.evtokcal*a.calc.Fstddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=False)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.3f Fsig = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev,Fstddev))
    
# Set the printer
dyn.attach(storeenergy, interval=50)

In [ ]:
start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(40000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')